In [1]:
import numpy as np
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
import sys
sys.path.append('/home/zhuf/CodingSpace/JupyterNotebook/machine_learning')
from playML.LinearRegression import LinearRegression
from playML.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [5]:
lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train, y_train)
lin_reg1.score(X_test, y_test)

CPU times: user 779 µs, sys: 108 µs, total: 887 µs
Wall time: 916 µs


0.8129794056212918

#  梯度下降法向量化以及直接使用梯度下降法的问题

在真实数据场景中，由于数据间的量纲差异比较大，会影响梯度的计算，进而使得步长太大而溢出，无法收敛

In [6]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train)

/home/zhuf/CodingSpace/JupyterNotebook/machine_learning/playML/LinearRegression.py:93: RuntimeWarning: overflow encountered in matmul
  return (tmp@tmp)/len(Xb)
/home/zhuf/CodingSpace/JupyterNotebook/machine_learning/playML/LinearRegression.py:112: RuntimeWarning: invalid value encountered in double_scalars
  if(abs(J(theta, Xb, y) - J(last_theta, Xb, y)) < epsilon):
/home/zhuf/CodingSpace/JupyterNotebook/machine_learning/playML/LinearRegression.py:99: RuntimeWarning: overflow encountered in matmul
  return Xb.T@(Xb@theta - y)*2/len(y)
/home/zhuf/CodingSpace/JupyterNotebook/machine_learning/playML/LinearRegression.py:92: RuntimeWarning: invalid value encountered in matmul
  tmp = y - Xb@theta
/home/zhuf/CodingSpace/JupyterNotebook/machine_learning/playML/LinearRegression.py:99: RuntimeWarning: invalid value encountered in matmul
  return Xb.T@(Xb@theta - y)*2/len(y)


LinearRegression_MyOwnVersion

In [7]:
lin_reg2.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

为了减少步长，可以减小学习率，但是效果依然不理想

In [8]:
lin_reg2.fit_gd(X_train, y_train, eta=0.0000001)

LinearRegression_MyOwnVersion

In [9]:
lin_reg2.score(X_test, y_test)

0.09905050111174063

为了尽量使得搜索收敛到最小值，在减小步长的同时，还增加搜索次数，但是这时候也非常耗时了

In [10]:
%time lin_reg2.fit_gd(X_train, y_train, eta=0.0000001, n_iters=1e6)

CPU times: user 24.1 s, sys: 13.7 ms, total: 24.1 s
Wall time: 25.3 s


LinearRegression_MyOwnVersion

但是效果依然不是很理想

In [11]:
lin_reg2.score(X_test, y_test)

0.5183036098651662

# 使用梯度下降法需要进行数据归一化

In [12]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)

数据归一化后，精度明显提升，效率也明显提升

In [13]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

CPU times: user 112 ms, sys: 4 µs, total: 112 ms
Wall time: 112 ms


LinearRegression_MyOwnVersion

In [14]:
X_test_standard = standardScaler.transform(X_test)
lin_reg3.score(X_test_standard, y_test)

0.8129873310487505

# 梯度下降法的优势

MOOC老师讲解错误，这个似乎没有体现出梯度下降法的优势，当m<n的时候， 正规方程的解貌似不正确，无法求得唯一解
当m>n的时候，两个解都一样了，但是正规方程解又明显快于梯度下降法

In [15]:
m = 5000
n = 3000

big_X = np.random.normal(size=(m, n))

true_theta = np.random.uniform(0.0, 100.0, size=n+1)

big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10., size=m)

In [16]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

CPU times: user 5.16 s, sys: 190 ms, total: 5.35 s
Wall time: 5.62 s


LinearRegression_MyOwnVersion

In [17]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y, eta=0.1)

CPU times: user 39 s, sys: 71.9 ms, total: 39.1 s
Wall time: 41 s


LinearRegression_MyOwnVersion

In [18]:
true_theta

array([21.19277475, 51.18666003, 16.15939272, ..., 33.16395395,
       79.43259013, 99.36657898])

In [21]:
big_reg1._theta

array([21.10630483, 51.42361339, 16.28807373, ..., 33.07710934,
       79.41034471, 99.20162212])

In [22]:
big_reg2._theta

array([21.10630938, 51.42354602, 16.28797583, ..., 33.07713666,
       79.41045667, 99.20154211])

## 线性回归正规方程的不可逆性

有时候$X^T_bX_b$是不可逆的（奇异矩阵）。造成不可逆的原因通常是以下两个：

- 有冗余特征： 有两个特征之间存在着一定联系，比如一个单位x1是米，另一个x2单位是千米，但表示的是同一个特征，这时候这两个特征之间存在着关系x2=x1。 根据线性代数的知识， 线形相关的矩阵不可逆的。
- 特征数n>训练集数m